In [56]:
import cv2
import numpy as np

In [57]:
img = cv2.imread(".\\data\\aadhar_demo2.jpg")

# Load the EAST Model

In [58]:
model_path = ".\\models\\frozen_east_text_detection.pb"
net = cv2.dnn.readNet(model_path)
if net.empty():
    raise Exception("Model not loaded, check the path!")

In [59]:
# Use multiple of 32 to set the new image shape
height, width, _ = img.shape
new_height = (height//32)*32
new_width = (width//32)*32
print(new_height, new_width)

384 640


In [60]:
# Get the ratio change in width and height
h_ratio = height / new_height
w_ratio = width / new_width
print(h_ratio, w_ratio)

1.0208333333333333 1.0


In [61]:
blob = cv2.dnn.blobFromImage(img, 1, (new_width, new_height), (123.68, 116.78, 103.94), True, False)

In [62]:
print(cv2.__version__)

4.12.0


# Pass the image to network and extract score and geometry map

In [63]:
net.setInput(blob)

In [64]:
net.getUnconnectedOutLayersNames()

('feature_fusion/concat_3', 'feature_fusion/Conv_7/Sigmoid')

In [65]:
(geometry, scores) = net.forward(net.getUnconnectedOutLayersNames())

# Post-Processing

In [66]:
rectangles = []
confidence_score = []
for i in range(geometry.shape[2]):
    for j in range(0, geometry.shape[3]):

        if scores[0][0][i][j] < 0.1:
            continue

        bottom_x = int(j*4 + geometry[0][1][i][j])
        bottom_y = int(i*4 + geometry[0][2][i][j])


        top_x = int(j*4 - geometry[0][3][i][j])
        top_y = int(i*4 - geometry[0][0][i][j])

        rectangles.append((top_x, top_y, bottom_x, bottom_y))
        confidence_score.append(float(scores[0][0][i][j]))

In [67]:
from imutils.object_detection import non_max_suppression

In [68]:
# use Non-max suppression to get the required rectangles
fin_boxes = non_max_suppression(np.array(rectangles), probs=confidence_score, overlapThresh=0.5)

In [70]:
img_copy = img.copy()
for (x1, y1, x2, y2) in fin_boxes:

    x1 = int(x1 * w_ratio)
    y1 = int(y1 * h_ratio)
    x2 = int(x2 * w_ratio)
    y2 = int(y2 * h_ratio)

    cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

In [72]:
cv2.imshow("original Image", img)
cv2.imshow("Text Detection", img_copy)
cv2.waitKey(0)

-1